## Unsupervised learning and EDA

Content here is licensed under a CC 4.0 License. The code in this notebook is released under the MIT license. 

By Manu Flores. 

In [ ]:
# uncomment the next line if you're in Google Collab 
#! pip install -r https://raw.githubusercontent.com/manuflores/grnlearn_tutorial/master/requirements.txt
#! wget https://raw.githubusercontent.com/manuflores/grnlearn_tutorial/master/notebooks/grn.py

In [ ]:
import grn as g
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import mixture
from umap import UMAP

import hvplot
import hvplot.pandas
import holoviews as hv
from holoviews import dim, opts
import bokeh_catplot
import bokeh 
import bokeh.io
from bokeh.themes import Theme
from bokeh.io import output_file, save, output_notebook

output_notebook()
hv.extension('bokeh')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

g.set_plotting_style()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

seed = 8
np.random.seed(seed)

In [ ]:
theme = Theme(json=g.bokeh_style())
bokeh.io.curdoc().theme = theme
hv.renderer('bokeh').theme = theme

Hey, cool that you stuck around for the whole tutorial. If you're just arriving to the repo let me just summarize what we've done so far: we extracted a simple genetic network (the Purine metabolism network of *E. coli*), and we used data to learn its patterns, isn't that amazing ! We used a simple linear model to derive the accurately learn the probability that a given gene is inside (our not) regulated or coexpressed in/with this biological module. 

However, we didn't really performed exploratory data analysis on the dataset, and you've might've felt that it would be cool to have done so. Well, I actually would've wanted this notebook to be the second one, but in the end, the last one was the core of the tutorial so that's why I ended up decided putting it first. But now's the time. Remember when I said that a non-linear dimension reduction would be better for our dataset, well let's continue with that train of thought here. 

### Unsupervised learning : manifold learning and clustering. 

Biological systems are highly non-linear. Even a simple kinetic reaction already involves a non-linear term - and there are thousands of such reactions even in the simplest forms of life (sorry bacteria, I *know* you're not simple) like bacteria. In this sense a linear model like PCA would most of the times, fail to capture the inherent structure of a high-dimensional dataset like the RNAseq data we just analyzed. Moreover, many times, as data scientists we want *to just a feel for the data* by visualizing it. This is were non-linear dimension reduction methods for biology shine!

One of the most widely used of such methods is t-SNE. And yeah, there are a lot of biologists that are quite against the t-SNE plots and how they are making (computational) biologists dumb because we are not really seeking for the understanding of the biological system, but here I want to argue that while the latter is true, this techniques help us quite a lot if used correctly. For now I will leave the matter aside and just go ahead and used my favorite non-linear dim reduction method now: UMAP. If you want to get at the gory details of how this method works, I highly recommend this great blog post from [Niko Oskolkov](https://towardsdatascience.com/how-to-program-umap-from-scratch-e6eff67f55fe).


Moreover, another approach that we could've have taken to learn the structure of gene networks is an unsupervised one: don't impose any knowledge from the data, let the data speak for itself. In this sense this is a much more exploratory intensive road and one must be careful to investigate that the extracted structure has biological relevance but I also want to argue that this is actually a very very interesting approach (and the most commonly used) for biology. 


### Load in the data

Enough of the rant, let's proceed to load in our PCA-denoised data. 

In [ ]:
path = '../data/'

In [ ]:
regulons_with_noise = pd.read_csv(path + 'denoised_coli_palsson_data.csv')

In [ ]:
regulons_with_noise.head()

Let's extract the numerical values only.

In [ ]:
data = regulons_with_noise.iloc[:, 3:].values

We can now proceed to apply the UMAP method. Fortunately, there is a great implementation of UMAP in the [umap-learn](https://umap-learn.readthedocs.io/) library. It also has great documentation, tons of intuitive examples and just an awesome API. It also runs on [Numba](http://numba.pydata.org/), so it is super well optimized. This will take a bit of time, so just remember- patience.

In [ ]:
latent_space = UMAP(n_neighbors = 5, random_state = seed).fit_transform(data)

In [ ]:
latent_space.shape

Nice. We can see that as a default the UMAP returns a two dimensional latent space of the data but importantly this can scale to $k$-dimensions. Let's proceed to add this UMAP latent space into our data and visualize it with `hvplot`

In [ ]:
regulons_with_noise['UMAP 1'], regulons_with_noise['UMAP 2'] = latent_space[:, 0], latent_space[:, 1]

## Exploratory data analysis on the UMAP space

We can finally check if there is a structure to our dataset in the UMAP latent space. First, we'll define some plotting options and then we'll make a scatter plot in the UMAP 2D space. 

In [ ]:
regulons_with_noise.head()

In [ ]:
dots_kws = {'padding': 0.2,
            'alpha' : 0.3,
            'tools': ['hover'],
            'color' : 'orange',
            'show_grid': True, 
            'width': 420, 
            'height': 300,
            'size': 5}

In [ ]:
dots= hv.Points(data = regulons_with_noise,
           kdims = ['UMAP 1', 'UMAP 2'],
           vdims = ['gene_name']).opts(**dots_kws,
                                       
                                      xlabel= 'UMAP 1',
                                      ylabel = 'UMAP 2' )

dots

Interesting ! We can see that there is like a manifold that our data lies in. Now, let's try to find out what is the hidden variables in the manifold. 

### Clustering and extracting data. 

Because of the structure of the latenst space, a good algorithm to try would be one that is based on density (e.g. DBSCAN) or that can extract nonlinear features to cluster. 

A perhaps even simpler approach is to just try to approximate the dataset using a mixture of 2D Gaussian distributions, i.e. a Gaussian mixture model. This algorithm is strikingly fast because it uses variational inference and converges almost instantly. 

One thing to do before is to choose how many clusters we want to extract from the data, i.e. how many Gaussians does the dataset actually encode. We can use the relevant biological information from the analysisi in the first notebook: the TRN can be partitioned into 16 clusters. Let's use this for for our number of components. 

In [ ]:
# Fit a Dirichlet process Gaussian mixture 
dpgmm = mixture.BayesianGaussianMixture(n_components=16,
                                        covariance_type='full', 
                                        random_state = seed).fit(latent_space)

Because the GMM is a probabilistic generative model, we can calculate the probabililty that each gene is a member of each cluster. This let's us assign multiple clusters to a given gene if we wanted to.

In [ ]:
probs = dpgmm.predict_proba(latent_space)

Let's look at the output to see what's the probability of the first 5 genes to be in the first four clusters. 

In [ ]:
probs[:5, :4]

Now, let's use the `predict`method to get the most likely clusters each gene belongs to.

In [ ]:
labels = dpgmm.predict(latent_space)

We can also check how many genes are in each cluster.

In [ ]:
pd.Series(labels).value_counts()

Now let's add the labels to visualize our clusters in the latent space. 

In [ ]:
regulons_with_noise['cluster_labels'] = labels

In [ ]:
regulons_with_noise.hvplot(kind = 'scatter',
                           x = 'UMAP 1',
                           y = 'UMAP 2',
                           c = 'cluster_labels',
                           hover_cols = ['gene name'],
                           s = 80, alpha = 0.1).opts(cmap = 'magma',
                                                      padding = 0.5,
                                                      height = 350, 
                                                      width = 500,
                                                      colorbar_opts={'title':'clusters'})

Nice! From exploring the data we can see that cluster 4 and cluster 10 are the furthest appart in the manifold. Let's check if there's an enriched function for each of this datasets. 

In [ ]:
# extract the data from cluster 11 and 6
c6 = regulons_with_noise[regulons_with_noise['cluster_labels'] == 6]
c11 = regulons_with_noise[regulons_with_noise['cluster_labels'] == 11]

In [ ]:
c6.head()

In [ ]:
# extract the genes from cluster 4 and 10 a
c6_genes = c6['gene_name'].values
c11_genes = c11['gene_name'].values

### Enriched biological functions. 

A normal approach to find the enriched or overrepresented functions in a set of genes, in our case, the clusters in the UMAP space, is to make a statistical test. The story is that we can approximate genome as a *bag of gene names*, therefore the probability to get a drawing a number of genes from a given biological module (say of the fatty acid metabolic pathway) from this bag is hypergeometrically distributed. Don't worry if you don't get the gist of the statistical story, but essentially we are going to test if a given biological function is enriched in our clusters. 

To get the annotations I got the *E. coli* annotation data from [Gene Ontology]('http://geneontology.org/') which is kind of a gold standard of biological function annotations. The full annotation can be found [here](https://zenodo.org/record/3552960).

I also made a wrapper function to call make the test on python. It's still work in progress but it will be good for now. 

In [ ]:
# Run the enrichment test on the genes of cluster 6
go_test_c6 = g.get_GO_enrichment(c6_genes)

In [ ]:
go_test_c6

Interesting ! We can see that the functions enriched appear to be metal binding enzymes. Don't pay too much attention to the p-values, I still have to work on correcting them. 

In [ ]:
# Same for cluster 11
go_test_c11 = g.get_GO_enrichment(c11_genes)

It looks like there are no statistically enriched functions for cluster 11. 

Hmm. It seems that we didn't got too much information to go on to the tips of the latent space. Let's see if we have any luck by going into the center of it. 

In [ ]:
c0 = regulons_with_noise[regulons_with_noise['cluster_labels'] == 0]
c0_genes = c0['gene_name'].values
go_test_c0 = g.get_GO_enrichment(c0_genes)

In [ ]:
go_test_c0

Interesting, it looks like we got a bunch of endonucleases and DNA repairing enzymes. 

Don't think that I'm a biological / *E. coli* genie, I just looked for information at [Ecocyc](https://ecocyc.org/). In this sense, I can be pretty sure that the clusters derived from the UMAP latent space are yielding biologically meaningful relationships. Exploring this dataset could take quite some time (it was two full years of my undergrad thesis), so I'll stop here and let you continue with the analysis if you're interested. 



### Another idea for post- clustering analysis

Another interesting idea would be to see if there are related transcription factors in each cluster. You now have the tools to answer this question!


### Last words

This was a fun ride in biological data analysis! I just want to acknowledge that most of this tutorial was based on work I did in the Pérez-Rueda Lab. 

Let me know if you have any questions and feel free to reach out if you want to collaborate in extending these ideas! 

**GRN** = **G**ene **R**egulatory **N**etwork
